<a href="https://colab.research.google.com/github/ArkanDash/Advanced-RVC-Inference/blob/master/Advanced-RVC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced RVC Inference

A modular Retrieval-based Voice Conversion framework with Gradio UI, training capabilities, and audio processing tools

[Discord](https://discord.gg/hvmsukmBHE) - [Github](https://github.com/ArkanDash/Advanced-RVC-Inference)

In [ ]:
# @title Setup runtime environment
# @markdown ### Choose installation method:
install_method = "local"  # @param ["local", "git+uv", "git+pip"]
from IPython.display import clear_output
import os

# Install system dependencies for audio processing
!apt-get -y install libportaudio2 ffmpeg git -qq > /dev/null 2>&1

# Install uv for faster package management
!pip install uv -q

match install_method:

  case 'local':
    # Clone from git and install locally (Recommended for development)
    if os.path.exists("Advanced-RVC-Inference"):
        %cd Advanced-RVC-Inference
        !git pull origin master
    else:
        !git clone https://github.com/ArkanDash/Advanced-RVC-Inference.git
        %cd Advanced-RVC-Inference
    
    # Install from requirements.txt
    !uv pip install -r requirements.txt --system -q
    
    # Install package in editable mode
    !uv pip install -e . --system -q
    
  case 'git+uv':
    # Install latest version from GitHub with uv (fastest)
    !uv pip install git+https://github.com/ArkanDash/Advanced-RVC-Inference.git -q
  
  case 'git+pip':
    # Install latest version from GitHub with pip
    !pip install git+https://github.com/ArkanDash/Advanced-RVC-Inference.git -q

clear_output()
print("Finished installing Advanced RVC Inference!")
print("Use the cell below to start the server.")

In [ ]:
# @title **Start server**
# @markdown ### Choose a sharing method:
method = "gradio"  # @param ["gradio", "localtunnel", "ngrok"]
ngrok_token = ""  # @param {type:"string"}
from IPython.display import clear_output
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Change to project directory if using local installation
if install_method == 'local':
    repo_dir = "Advanced-RVC-Inference"
    # Check if we're already in the repo directory
    if os.path.basename(os.getcwd()) != repo_dir:
        if os.path.exists(repo_dir):
            %cd $repo_dir
            !git pull -q

clear_output()

match method:

  case 'gradio':
    !rvc-gui --share

  case 'localtunnel':
    !pip install localtunnel -q
    !echo "Password IP: $(curl --silent https://ipv4.icanhazip.com)"
    !echo
    !lt --port 7869 & rvc-gui --listen --server-name 0.0.0.0

  case 'ngrok':
    !pip install pyngrok -q
    from pyngrok import ngrok
    ngrok.kill()
    if ngrok_token:
        ngrok.set_auth_token(ngrok_token)
    public_url = ngrok.connect(7869)
    print(f"Ngrok URL: {public_url}")
    !rvc-gui --listen --server-name 0.0.0.0 --server-port 7869